In [1]:
import os

names = os.listdir(r"C:\Users\Derin\Desktop\image_png - Copy (2)")



In [187]:
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import matplotlib
matplotlib.style.use('ggplot')
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import pandas as pd

names_df = pd.DataFrame({'Name':[x.split('.')[0] for x in names], 
                         'Path':[r"C:\Users\Derin\Desktop\image_png - Copy (2)" + "\\" + x for x in names]})


In [3]:
product_df2 = pd.read_csv(r"C:\Users\Derin\Desktop\bitirme_Data - Copy.csv")


In [4]:
merged_df = pd.merge(left=names_df, right=product_df2, on='Name', how='inner')

merged_df

,Name,Path,Label
0,Cookplus 4251 Festymix El Mikseri Retro Kirmizi,C:\Users\Derin\Desktop\image_png - Copy (2)\Co...,3
1,Cookplus Blue Tasinabilir Sarili Smoothie Blender,C:\Users\Derin\Desktop\image_png - Copy (2)\Co...,3
2,Cookplus Chopper Dograyici Xxl 4001 Pink,C:\Users\Derin\Desktop\image_png - Copy (2)\Co...,3
3,Cookplus Chopper Dograyici Xxl 4001 Retro Kirmizi,C:\Users\Derin\Desktop\image_png - Copy (2)\Co...,3
4,Cookplus Coffee Keyf Filtre Kahve Makinesi Ino...,C:\Users\Derin\Desktop\image_png - Copy (2)\Co...,0
...,...,...,...
433,Smeg Narenciye Sikacagi Red Cjf01rdeu,C:\Users\Derin\Desktop\image_png - Copy (2)\Sm...,0
434,Smeg Narenciye Sikacagi White Cjf01wheu,C:\Users\Derin\Desktop\image_png - Copy (2)\Sm...,0
435,Smeg Smf03bleu Hamur Karistirma Makinesi Full ...,C:\Users\Derin\Desktop\image_png - Copy (2)\Sm...,3
436,Smeg Smf03rdeu Hamur Karistirma Makinesi Full Red,C:\Users\Derin\Desktop\image_png - Copy (2)\Sm...,3


In [191]:
from PIL import Image
from numpy import asarray

In [192]:
shuffled_df = merged_df.sample(frac=1).reset_index(drop=True)

train_df, val_df, test_df = shuffled_df[:361], shuffled_df[361:402], shuffled_df[402:439]

In [193]:
def preprocess(im):

    im = cv2.imread(im, 1)
    im= cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im=cv2.resize(im, (600,600))
    im.resize(600, 600, 3)
     
    return im

In [194]:
import numpy as np
import cv2
test_pic, test_stats = [], []
test_y = []
for i, row in test_df.iterrows():
  picture_path = row['Path']
  stat_path    = row['Name'] 
  label_path   = row['Label']  
  
  test_pic.append(preprocess(picture_path))
  test_stats.append(stat_path)
  test_y.append(label_path)      

In [195]:
test_pic=np.array(test_pic)
test_stats=np.array(test_stats)
test_y=np.array(test_y)


In [196]:
import numpy as np
import cv2
val_pic, val_stats = [], []
val_y = []
for i, row in val_df.iterrows():
  picture_path = row['Path']
  stat_path    = row['Name'] 
  label_path   = row['Label']  
  
  val_pic.append(preprocess(picture_path))
  val_stats.append(stat_path)
  val_y.append(label_path)      

In [197]:
val_pic=np.array(val_pic)
val_stats=np.array(val_stats)
val_y=np.array(val_y)

In [198]:
import numpy as np
import cv2
train_pic, train_stats = [], []
train_y = []
for i, row in train_df.iterrows():
  picture_path = row['Path']
  stat_path    = row['Name'] 
  label_path   = row['Label']  
  
  train_pic.append(preprocess(picture_path))
  train_stats.append(stat_path)
  train_y.append(label_path)      

In [199]:
train_pic=np.array(train_pic)
train_stats=np.array(train_stats)
train_y=np.array(train_y)

In [200]:
train_pic = train_pic/255
val_pic   = val_pic/255
test_pic  = test_pic/255

In [236]:
def build_model(num_classes):
    model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=8, kernel_size=(3, 3), activation='relu', 
                           input_shape=(600, 600, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),        
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

model = build_model(num_classes=4)

In [237]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
print(model.summary())

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_171 (Conv2D)          (None, 598, 598, 8)       224       
_________________________________________________________________
max_pooling2d_172 (MaxPoolin (None, 299, 299, 8)       0         
_________________________________________________________________
conv2d_172 (Conv2D)          (None, 297, 297, 64)      4672      
_________________________________________________________________
max_pooling2d_173 (MaxPoolin (None, 148, 148, 64)      0         
_________________________________________________________________
conv2d_173 (Conv2D)          (None, 146, 146, 64)      36928     
_________________________________________________________________
max_pooling2d_174 (MaxPoolin (None, 73, 73, 64)        0         
_________________________________________________________________
conv2d_174 (Conv2D)          (None, 71, 71, 128)     

In [238]:
model.fit(train_pic, train_y, validation_data=(val_pic,val_y) ,epochs=20)


Epoch 1/20
12/12 [==============================] - 40s 3s/step - loss: 1.6553 - accuracy: 0.3850 - val_loss: 1.2411 - val_accuracy: 0.2195
Epoch 2/20
12/12 [==============================] - 38s 3s/step - loss: 1.2237 - accuracy: 0.4266 - val_loss: 1.0728 - val_accuracy: 0.7073
Epoch 3/20
12/12 [==============================] - 38s 3s/step - loss: 1.0917 - accuracy: 0.5596 - val_loss: 0.8338 - val_accuracy: 0.7073
Epoch 4/20
12/12 [==============================] - 38s 3s/step - loss: 0.9133 - accuracy: 0.6066 - val_loss: 0.6917 - val_accuracy: 0.7317
Epoch 5/20
12/12 [==============================] - 38s 3s/step - loss: 0.7652 - accuracy: 0.6953 - val_loss: 0.6059 - val_accuracy: 0.8293
Epoch 6/20
12/12 [==============================] - 38s 3s/step - loss: 0.6540 - accuracy: 0.7202 - val_loss: 0.5388 - val_accuracy: 0.8049
Epoch 7/20
12/12 [==============================] - 38s 3s/step - loss: 0.5479 - accuracy: 0.7812 - val_loss: 0.6293 - val_accuracy: 0.7805
Epoch 8/20
12/12 [==

In [240]:
model.evaluate(test_pic, test_y)


2/2 [==============================] - 0s 50ms/step - loss: 1.6939 - accuracy: 0.7778


[1.693901777267456, 0.7777777910232544]

In [220]:
training_sentences = train_stats
validation_sentences = val_stats
testing_sentences = test_stats

training_labels = train_y
validation_labels = val_y
testing_labels = test_y

In [221]:
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [222]:
vocab_size = 50
embedding_dim = 64
max_length = 10
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

validation_sequences = tokenizer.texts_to_sequences(validation_sentences)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [223]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(32, activation='softmax'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
]) 
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True), optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_52 (Embedding)     (None, 10, 64)            3200      
_________________________________________________________________
conv1d_53 (Conv1D)           (None, 6, 128)            41088     
_________________________________________________________________
global_average_pooling1d_56  (None, 128)               0         
_________________________________________________________________
dense_286 (Dense)            (None, 16)                2064      
_________________________________________________________________
dense_287 (Dense)            (None, 32)                544       
_________________________________________________________________
dense_288 (Dense)            (None, 64)                2112      
_________________________________________________________________
dense_289 (Dense)            (None, 4)               

In [228]:
num_epochs = 20
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(validation_padded, validation_labels))

Epoch 1/20
12/12 [==============================] - 0s 5ms/step - loss: 0.8772 - accuracy: 0.9030 - val_loss: 0.8964 - val_accuracy: 0.8780
Epoch 2/20
12/12 [==============================] - 0s 3ms/step - loss: 0.8662 - accuracy: 0.9030 - val_loss: 0.8883 - val_accuracy: 0.8780
Epoch 3/20
12/12 [==============================] - 0s 3ms/step - loss: 0.8608 - accuracy: 0.9058 - val_loss: 0.8854 - val_accuracy: 0.8780
Epoch 4/20
12/12 [==============================] - 0s 3ms/step - loss: 0.8531 - accuracy: 0.9086 - val_loss: 0.8916 - val_accuracy: 0.8780
Epoch 5/20
12/12 [==============================] - 0s 3ms/step - loss: 0.8529 - accuracy: 0.9086 - val_loss: 0.8776 - val_accuracy: 0.8780
Epoch 6/20
12/12 [==============================] - 0s 3ms/step - loss: 0.8503 - accuracy: 0.9058 - val_loss: 0.8919 - val_accuracy: 0.8537
Epoch 7/20
12/12 [==============================] - 0s 3ms/step - loss: 0.8479 - accuracy: 0.9086 - val_loss: 0.8754 - val_accuracy: 0.8780
Epoch 8/20
12/12 [==

In [229]:
model.evaluate(testing_padded, testing_labels,verbose=1)

2/2 [==============================] - 0s 500us/step - loss: 0.9487 - accuracy: 0.7778


[0.9486852288246155, 0.7777777910232544]

In [230]:
from tensorflow.keras.models import Model

In [241]:
inputs_im= tf.keras.layers.Input(shape=(600,600,3))
x     = tf.keras.layers.Conv2D(filters=8, kernel_size=(3, 3), activation='relu', 
                           input_shape=(600, 600, 3))(inputs_im)
x     = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2)(x)
x     = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x     = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2)(x)
x     = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x     = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2)(x)
x     = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(x)
x     = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2)(x)
x     = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(x)
x     = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2)(x)
x     =tf.keras.layers.Flatten()(x)
x     =tf.keras.layers.Dense(256, activation='softmax')(x)
x     =tf.keras.layers.Dense(4, activation='softmax')(x)
x     =tf.keras.Model(inputs=inputs_im, outputs=x)


inputs_word =tf.keras.layers.Input(shape=(max_length))
y           =tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length)(inputs_word)
y           =tf.keras.layers.Conv1D(128, 5, activation='relu')(y)
y           =tf.keras.layers.GlobalAveragePooling1D()(y)
y           =tf.keras.layers.Dense(16, activation='relu')(y)
y           =tf.keras.layers.Dense(32, activation='relu')(y)
y           =tf.keras.layers.Dense(64, activation='relu')(y)
y           =tf.keras.layers.Dense(4, activation='softmax')(y)
y           =tf.keras.Model(inputs=inputs_word, outputs=y)


combined =tf.keras.layers.concatenate([x.output,y.output])
z        =tf.keras.layers.Dense(16, activation="relu")(combined)
z        =tf.keras.layers.Dense(32, activation="relu")(z)
z        =tf.keras.layers.Dense(64, activation="relu")(z)
z        =tf.keras.layers.Dense(4, activation="softmax")(z)


model = tf.keras.Model(inputs=[x.input, y.input], outputs=z)
model.summary()

Model: "functional_203"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_77 (InputLayer)           [(None, 600, 600, 3) 0                                            
__________________________________________________________________________________________________
conv2d_176 (Conv2D)             (None, 598, 598, 8)  224         input_77[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_177 (MaxPooling2D (None, 299, 299, 8)  0           conv2d_176[0][0]                 
__________________________________________________________________________________________________
conv2d_177 (Conv2D)             (None, 297, 297, 64) 4672        max_pooling2d_177[0][0]          
_____________________________________________________________________________________

In [242]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [248]:
model.fit([train_pic,training_padded],train_y, validation_data=([val_pic,validation_padded],val_y) ,epochs=20)


Epoch 1/20
12/12 [==============================] - 40s 3s/step - loss: 0.2824 - accuracy: 0.8809 - val_loss: 0.4299 - val_accuracy: 0.8537
Epoch 2/20
12/12 [==============================] - 39s 3s/step - loss: 0.2517 - accuracy: 0.8920 - val_loss: 0.5010 - val_accuracy: 0.8293
Epoch 3/20
12/12 [==============================] - 40s 3s/step - loss: 0.2430 - accuracy: 0.8920 - val_loss: 0.5308 - val_accuracy: 0.8293
Epoch 4/20
12/12 [==============================] - 38s 3s/step - loss: 0.2455 - accuracy: 0.8975 - val_loss: 0.5697 - val_accuracy: 0.8537
Epoch 5/20
12/12 [==============================] - 38s 3s/step - loss: 0.2323 - accuracy: 0.9030 - val_loss: 0.4472 - val_accuracy: 0.8293
Epoch 6/20
12/12 [==============================] - 38s 3s/step - loss: 0.2173 - accuracy: 0.9141 - val_loss: 0.6166 - val_accuracy: 0.8537
Epoch 7/20
12/12 [==============================] - 38s 3s/step - loss: 0.2020 - accuracy: 0.9169 - val_loss: 0.5173 - val_accuracy: 0.8293
Epoch 8/20
12/12 [==

In [249]:
model.evaluate([test_pic,testing_padded], test_y,)


2/2 [==============================] - 0s 50ms/step - loss: 0.6980 - accuracy: 0.8611


[0.6980069279670715, 0.8611111044883728]